In [1]:
from gwosc.datasets import event_gps
from gwpy.timeseries import TimeSeries, TimeSeriesDict
from gwpy.frequencyseries import FrequencySeries
from gwpy.signal import filter_design
from gwpy.plot import Plot
from gwpy.signal import filter_design
from scipy.special import erf
from scipy.signal.windows import tukey
from ipywidgets import interactive_output, VBox, HBox, FloatSlider, Checkbox, FloatLogSlider, FloatRangeSlider, FloatText
import sounddevice as sd

import numpy as np
import matplotlib.pyplot as plt
import plotfancy as pf
pf.housestyle_rcparams()

True

In [2]:
def lrm(data, base_window=10, log_base=10):
    data = np.asarray(data)
    is_1d = False

    if data.ndim == 1:
        data = data[:, np.newaxis]
        is_1d = True

    n, m = data.shape
    mean = np.zeros_like(data)
    std = np.zeros_like(data)

    for i in range(n):
        window_size = max(1, int(np.floor(np.log(i + 1) / np.log(log_base)) * base_window))
        start = max(0, i - window_size + 1)
        window = data[start:i + 1]
        mean[i] = window.mean(axis=0)
        std[i] = window.std(axis=0, ddof=0)  # population std (ddof=0), change to ddof=1 if sample std

    if is_1d:
        return mean.ravel(), std.ravel()
    return np.array([mean, std])

In [3]:
# gps = event_gps('GW150914')
# data = TimeSeries.fetch_open_data('L1', gps-5, gps+5)
data = TimeSeries.fetch_open_data('H1', 1126259446,1126259478)

In [4]:
def gs(x, centre=0, width=1, softness=0.1):
    return 0.5 * (erf((x - (centre - width/2)) / softness) - erf((x - (centre + width/2)) / softness))

def gslr(x, left, right, softness=0.1):
    return 0.5 * (erf((x - (left)) / softness) - erf((x - (right)) / softness))


import numpy as np

def tukey_man(x: np.ndarray, left: float, right: float, softness: float) -> np.ndarray:
    if softness < 0 or softness > 1:
        raise ValueError("softness must be between 0 and 1")
    w = np.zeros_like(x)
    width = right - left
    if width <= 0:
        raise ValueError("right must be greater than left")
    taper = softness * width / 2
    start_taper_end = left + taper
    end_taper_start = right - taper
    mask_flat = (x >= start_taper_end) & (x <= end_taper_start)
    w[mask_flat] = 1.0
    mask_left = (x >= left) & (x < start_taper_end)
    w[mask_left] = 0.5 * (1 + np.cos(np.pi * (2 * (x[mask_left] - left) / (softness * width) - 1)))
    mask_right = (x > end_taper_start) & (x <= right)
    w[mask_right] = 0.5 * (1 + np.cos(np.pi * (2 * (x[mask_right] - right) / (softness * width) + 1)))
    return w

gridlims = [1,1000]
res = len(data.psd())

def process_and_plot(lr,s, notch_b, sound,notch_fun=60):
    grid = np.linspace(gridlims[0], gridlims[1], res)
    dat = tukey_man(grid, lr[0], lr[1],softness=s)

    bp = filter_design.bandpass(lr[0],lr[1], data.sample_rate) # 50-250hz bandpass
    mains = np.arange(1,4)*notch_fun
    twindow = tukey(len(data), alpha = s)
    if notch_b:
        notches = [filter_design.notch(line, data.sample_rate) for line in mains] # removing AC mains harmonics
        zpk = filter_design.concatenate_zpks(bp, *notches)
        dfilt = data.filter(zpk, filtfilt = True)
    else:
        dfilt = data.filter(bp, filtfilt = True)
    
    notch_f = notch_fun*np.arange(1,7)

    if notch_b:
        notch_mask = np.ones(len(grid))
        for notch in notch_f:
            notch_mask = notch_mask*(1-gs(grid, width=2, softness=0.1, centre=notch))
    
        dat = dat*notch_mask

    fig, ax1 = pf.create_plot((4,3))
    ax1.set_xlim(gridlims)
    ax1.set_ylim(-0.05,1.05)
    ax1.plot(grid,dat, color='black', lw=2, zorder=10)
    ax1.set_xscale('log')

    ax2 = ax1.twinx()
    ax2.plot(data.psd(), color='#77aca2', alpha=0.6, zorder=-10)
    ax2.set_yscale('log')
    ax2.set_ylabel('Noise PSD')

    ax1.set_ylabel('$\exp($Gain [dB]$)$')
    ax1.set_xlabel('Frequency [Hz]')
    pf.fix_plot([ax1,ax2])
    ax1.tick_params(top=True,right=False, direction='in', length=7, which='major')
    ax1.tick_params(top=True,right=False, direction='in', length=4, which='minor')

    ax3 = fig.add_axes((1.35,0,1,1))

    gw = dfilt.psd(window = twindow)
    gwrolling = lrm(gw)
    ax3.loglog(grid,gwrolling[0], color='#ff004f')
    # ax3.fill_between(grid,gwrolling[0]-gwrolling[1],gwrolling[0]+gwrolling[1], color='#ff004f', alpha=0.3)
    # ax3.set_ylim(5e-48, 5e-43)
    ax3.set_xlabel('Frequency [Hz]')

    ax4 = fig.add_axes((0,-1,2,.6))
    tgrid = np.arange(0,4,np.float64(data.dt))
    slength = int(len(dfilt))
    chop = 1000
    ax4.plot(np.array(data.times)[chop:(slength-chop)]-data.epoch.value,(dfilt*10**21)[chop:slength-chop], color='#ff004f')
    ax4.set_xlim([1,26])
    ax4.set_ylim([-1,1])
    ax4.set_ylabel('Strain')

    ax5 = fig.add_axes((0,-1.7,2,.6))
    ax5.plot(np.array(data.times)[chop:(slength-chop)]-data.epoch.value,(dfilt*10**21)[chop:slength-chop], color='#ff004f')
    ax5.set_xlim([16.04,16.64])
    ax5.set_ylim([-1,1])
    ax5.set_xlabel('Time [s]')
    ax5.set_ylabel('Strain')

    pf.fix_plot([ax3,ax4, ax5])
    plt.show()
    plt.close(fig)

    if sound:
        global time 
        time = np.array(data.times)-data.epoch.value
        global cond
        cond = (time>13)&(time<18)
        global dfilt_1sec
        dfilt_1sec = dfilt[cond]
        strain = np.array(dfilt_1sec) / np.array(np.max(np.abs(dfilt_1sec)))  # normalize
        sd.play(strain, samplerate=np.float64(data.sample_rate))
        sd.wait()

#### - OLD - ####
# l_slider = FloatLogSlider(min=0, max=3, step=0.01, value=np.log10(25), description='BP Left')
# r_slider = FloatLogSlider(min=0, max=3, step=0.01, value=np.log10(300), description='BP Right')

#### - SLIDERS - ####
lr_slider = FloatRangeSlider(min=1, max=1000, step=0.01, value=[50,250], description='BP')
s_slider = FloatLogSlider(min=-1, max=0, step=0.1, value=0.5, description='Softness')
n_slider = FloatLogSlider(min=0, max=3, step=0.001, value=60, description='Ground H0')
n_checkbox = Checkbox(value=True, description='Notches On?')
s_checkbox = Checkbox(value=False, description='Play Sound')

#### - PLOTTING - ####
out = interactive_output(process_and_plot, {'lr': lr_slider, 's':s_slider, 'notch_b':n_checkbox, 'notch_fun':n_slider, 'sound':s_checkbox})
VBox([lr_slider, s_slider,n_checkbox,n_slider, s_checkbox, out])



<>:65: SyntaxWarning: invalid escape sequence '\e'
<>:65: SyntaxWarning: invalid escape sequence '\e'
/var/folders/6b/x84gj0ds47n3hjglw1wq1j8m0000gn/T/ipykernel_5091/932528778.py:65: SyntaxWarning: invalid escape sequence '\e'
  ax1.set_ylabel('$\exp($Gain [dB]$)$')
